# Reading the Training File

In [84]:
train_file_path = './POS_CORPUS_FOR_STUDENTS/POS_train.pos'
dev_file_path = './POS_CORPUS_FOR_STUDENTS/POS_dev.words'
test_file_path = './POS_CORPUS_FOR_STUDENTS/'

### Calculating Emission Probabilities

In this section we create a dictionary with all of the words in the training set as keys and a counter of every POS associated with each word as values.

In [85]:
from collections import Counter
import numpy as np
from copy import copy

In [86]:
word_pos_dict = {}
pos_set = set()
with open(train_file_path, 'r') as file:
    for line in file:
        line = line.strip()
        if line:
            word, tag = line.split('\t')
            pos_set.add(tag)
            if word in word_pos_dict:
                if tag in word_pos_dict[word]:
                    word_pos_dict[word][tag] += 1
                else:
                    word_pos_dict[word][tag] = 1
            else:
                c = Counter()
                c[tag] = 1
                word_pos_dict[word] = c
                
                
pos_set = list(pos_set)
word_set = list(word_pos_dict)

In [87]:
word_pos_dict

{'mid-week': Counter({'NN': 1}),
 '1,800': Counter({'CD': 7}),
 'Poetry': Counter({'NNP': 1}),
 'Larsen': Counter({'NNP': 9}),
 'hanging': Counter({'NN': 1, 'VBG': 8}),
 'Hawaiian\\/Japanese': Counter({'JJ': 1}),
 'five-nation': Counter({'JJ': 1}),
 'comically': Counter({'RB': 1}),
 'localized': Counter({'JJ': 1, 'VBN': 1}),
 'Schuster': Counter({'NNP': 3}),
 '161.1': Counter({'CD': 2}),
 '161.3': Counter({'CD': 1}),
 'scold': Counter({'VB': 1}),
 'Archuleta': Counter({'NNP': 3}),
 '153.3': Counter({'CD': 1}),
 'refunding': Counter({'JJ': 3, 'NN': 4, 'VBG': 4}),
 'Western': Counter({'JJ': 66, 'NNP': 80}),
 '2.4375': Counter({'CD': 1}),
 'Rajiv': Counter({'NNP': 3}),
 'Euro': Counter({'NNP': 5}),
 'Apicella': Counter({'NNP': 1}),
 'wracked': Counter({'VBD': 1, 'VBN': 1}),
 'stipulate': Counter({'VB': 1}),
 'pigment': Counter({'NN': 4}),
 'appropriation': Counter({'NN': 3}),
 'Fridman': Counter({'NNP': 1}),
 'bringing': Counter({'NN': 1, 'VBG': 36}),
 'seamier': Counter({'JJR': 1}),
 'Go

In [88]:
num_words = len(word_set)
num_pos = len(pos_set)

emission_probs = np.ndarray(shape=(num_pos,num_words))

for pi in range(num_pos):
    for wi in range(num_words):
        
        word = word_set[wi]
            
        pos = pos_set[pi]

        word_poses = word_pos_dict[word]

        total = sum(word_poses.values())
            
        if pos in word_poses:
            emission_probs[pi,wi] = word_poses[pos]/float(total)
        else:
            emission_probs[pi,wi] = 0
            
            
        

In [89]:
emission_probs

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [90]:
lines = []
with open(train_file_path, 'r') as file:
    line_buf = []
    for line in file:
        line = line.strip()
        if line:
            _, tag = line.split('\t')
            line_buf.append(tag)

        else:
            lines.append(copy(line_buf))
            line_buf = []


pos_successor_dict = {}
for line in lines:
    for i in range(len(line)):
        if i < len(line)-1:
            next_pos = line[i+1]
            pos = line[i]
            if pos in pos_successor_dict:
                c = pos_successor_dict[pos]
                if next_pos in c:
                    c[next_pos] += 1
                else:
                    c[next_pos] = 1
            else:
                c = Counter()
                c[next_pos] = 1
                pos_successor_dict[pos] = c

                
transiton_probs = np.ndarray(shape=(num_pos, num_pos))
for i in range(num_pos):
    
    pos = pos_set[i]
    next_poses = pos_successor_dict[pos]
    total = sum(next_poses.values())
    
    for j in range(num_pos):
        
        next_pos = pos_set[j]
        if next_pos in next_poses:
            transiton_probs[i,j] = next_poses[next_pos]/float(total)
        else:
            transiton_probs[i,j] = 0

In [91]:
transiton_probs

array([[0.00000000e+00, 7.85060069e-03, 3.56845486e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.06479860e-02, 4.51300013e-03, 3.09847770e-03, ...,
        6.73582110e-05, 1.01037316e-03, 2.02074633e-04],
       [2.65007027e-02, 2.27531286e-02, 1.80686609e-03, ...,
        0.00000000e+00, 2.81068059e-03, 1.00381450e-04],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.38450480e-02, 1.75531512e-02, 1.28622229e-03, ...,
        0.00000000e+00, 4.91790875e-03, 1.13490202e-04],
       [1.03092784e-02, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 2.06185567e-02, 3.09278351e-02]])

In [92]:
def viterbi(y, A, B, Pi=None):
    """
    Return the MAP estimate of state trajectory of Hidden Markov Model.

    Parameters
    ----------
    y : array (T,)
        Observation state sequence. int dtype.
    A : array (K, K)
        State transition matrix. See HiddenMarkovModel.state_transition  for
        details.
    B : array (K, M)
        Emission matrix. See HiddenMarkovModel.emission for details.
    Pi: optional, (K,)
        Initial state probabilities: Pi[i] is the probability x[0] == i. If
        None, uniform initial distribution is assumed (Pi[:] == 1/K).

    Returns
    -------
    x : array (T,)
        Maximum a posteriori probability estimate of hidden state trajectory,
        conditioned on observation sequence y under the model parameters A, B,
        Pi.
    T1: array (K, T)
        the probability of the most likely path so far
    T2: array (K, T)
        the x_j-1 of the most likely path so far
    """
    # Cardinality of the state space
    K = A.shape[0]
    # Initialize the priors with default (uniform dist) if not given by caller
    Pi = Pi if Pi is not None else np.full(K, 1 / K)
    T = len(y)
    T1 = np.empty((K, T), 'd')
    T2 = np.empty((K, T), 'B')

    # Initilaize the tracking tables from first observation
    T1[:, 0] = Pi * B[:, y[0]]
    T2[:, 0] = 0

    # Iterate throught the observations updating the tracking tables
    for i in range(1, T):
        T1[:, i] = np.max(T1[:, i - 1] * A.T * B[np.newaxis, :, y[i]].T, 1)
        T2[:, i] = np.argmax(T1[:, i - 1] * A.T, 1)

    # Build the output, optimal model trajectory
    x = np.empty(T, 'B')
    x[-1] = np.argmax(T1[:, T - 1])
    for i in reversed(range(1, T)):
        x[i - 1] = T2[x[i], i]

    return x, T1, T2

In [112]:
transiton_probs

array([[0.00000000e+00, 7.85060069e-03, 3.56845486e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.06479860e-02, 4.51300013e-03, 3.09847770e-03, ...,
        6.73582110e-05, 1.01037316e-03, 2.02074633e-04],
       [2.65007027e-02, 2.27531286e-02, 1.80686609e-03, ...,
        0.00000000e+00, 2.81068059e-03, 1.00381450e-04],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.38450480e-02, 1.75531512e-02, 1.28622229e-03, ...,
        0.00000000e+00, 4.91790875e-03, 1.13490202e-04],
       [1.03092784e-02, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 2.06185567e-02, 3.09278351e-02]])

In [94]:
lines = []

#we are going to add unknown words to the list of words and emisions, 
#therefore take a copy so that we don't mess up the original 
words = copy(word_set)
emissions = emission_probs.copy()
equal_prob = np.full(shape=(len(pos_set), 1), fill_value=(1/float(len(pos_set))))
with open(dev_file_path, 'r') as file:
    line_buf = []
    for line in file:
        line = line.strip()
        if line:
            if line in word_set:
                line_buf.append(words.index(line))
            else:
                # word not seen in training data, append it to the set of words with equal probability 
                words.append(line)
                emissions = np.hstack([emissions, equal_prob])
        else:
            lines.append(np.array(line_buf))
            line_buf = []

In [105]:
path, _, _ = viterbi(lines[2], transiton_probs, emissions)

array([0.00000000e+00, 7.85060069e-03, 3.56845486e-04, 6.42321875e-03,
       2.37896991e-04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.40632806e-01, 0.00000000e+00, 1.18948495e-04, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 7.13690972e-03, 4.45343166e-01,
       3.56845486e-04, 9.51587962e-04, 3.56845486e-04, 0.00000000e+00,
       1.18948495e-04, 0.00000000e+00, 0.00000000e+00, 5.47163078e-03,
       1.18948495e-04, 1.94837635e-01, 4.85309861e-02, 4.87688831e-03,
       0.00000000e+00, 5.94742477e-04, 0.00000000e+00, 0.00000000e+00,
       2.02212442e-03, 1.18948495e-04, 1.97454502e-02, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 5.94742477e-04,
       1.02295706e-02, 2.97371238e-03, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00])

In [107]:
[pos_set[i] for i in path]

['PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$',
 'PRP$']

In [104]:
[word_set[i] for i in lines[0]]

['The',
 'economy',
 "'s",
 'temperature',
 'will',
 'be',
 'taken',
 'from',
 'several',
 'vantage',
 'points',
 'this',
 'week',
 ',',
 'with',
 'readings',
 'on',
 'trade',
 ',',
 'output',
 ',',
 'housing',
 'and',
 'inflation',
 '.']